In [3]:
import pandas as pd
import re
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

# General overview

In [3]:
df = pd.read_excel('/Users/jennyhuang/Documents/jobs/DBC-AI Launch/project2-MDdataCleaning/data_cleaning_and_analysis.xlsx')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158 entries, 0 to 157
Data columns (total 12 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   provider-name__lnk               157 non-null    object
 1   provider-details__specialty      157 non-null    object
 2   sr-only                          157 non-null    object
 3   star-rating__reviews             157 non-null    object
 4   sr-only 2                        124 non-null    object
 5   location-info__office-loc        157 non-null    object
 6   location-info__office-loc 2      157 non-null    object
 7   feats-of-strength__feat-title    120 non-null    object
 8   feats-of-strength__feat-title 2  106 non-null    object
 9   feats-of-strength__feat-title 3  104 non-null    object
 10  location-info__distance 2        157 non-null    object
 11  star-rating href                 33 non-null     object
dtypes: object(12)
memory usage: 14.9+ KB

In [5]:
df.head()

,provider-name__lnk,provider-details__specialty,sr-only,star-rating__reviews,sr-only 2,location-info__office-loc,location-info__office-loc 2,feats-of-strength__feat-title,feats-of-strength__feat-title 2,feats-of-strength__feat-title 3,location-info__distance 2,star-rating href
0,"Dr. Joseph Fata, MD","Cosmetic, Plastic & Reconstructive Surgery","Rated 4.5 out of 5 stars,",51 ratings,with,10293 N Meridian St Ste 200,"Indianapolis, IN 46290",Offers Telehealth,Easy scheduling,Employs friendly staff,11.5 mi,NaN
1,"Dr. Janet Turkle, MD","Cosmetic, Plastic & Reconstructive Surgery","Rated 5 out of 5 stars,",303 ratings,with,11455 N Meridian St Ste 150,"Carmel, IN 46032",Easy scheduling,Employs friendly staff,Explains conditions well,12.8 mi,NaN
2,"Dr. Emily Hrisomalos, MD","Cosmetic, Plastic & Reconstructive Surgery","Rated 5 out of 5 stars,",69 ratings,with,11455 N Meridian St Ste 150,"Carmel, IN 46032",Easy scheduling,Employs friendly staff,Explains conditions well,12.8 mi,NaN
3,"Dr. Christopher Jones, MD","Cosmetic, Plastic & Reconstructive Surgery","Rated 5 out of 5 stars,",39 ratings,with,320 N Meridian St,"Indianapolis, IN 46204",Easy scheduling,Employs friendly staff,Explains conditions well,0.2 mi,NaN
4,"Dr. Rachel Scott, MD","Cosmetic, Plastic & Reconstructive Surgery","Rated 5 out of 5 stars,",25 ratings,with,100 Hospital Ln,"Danville, IN 46122",Offers Telehealth,Easy scheduling,Employs friendly staff,18.3 mi,NaN


In [6]:
file = pd.ExcelFile('/Users/jennyhuang/Documents/jobs/DBC-AI Launch/project2-MDdataCleaning/data_cleaning_and_analysis.xlsx')

In [7]:
names = file.sheet_names
names

['Indiana', 'Missouri', 'Iowa', 'Wisconsin', 'Nebraska', 'Kansas', 'Illinois']

In [8]:
sheet = {}
for i in range(len(names)):
    sheet[names[i]] = file.parse(i, header=None)

In [9]:
for i in range(len(sheet)):
    sheet[names[i]].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159 entries, 0 to 158
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       158 non-null    object
 1   1       158 non-null    object
 2   2       158 non-null    object
 3   3       158 non-null    object
 4   4       125 non-null    object
 5   5       158 non-null    object
 6   6       158 non-null    object
 7   7       121 non-null    object
 8   8       107 non-null    object
 9   9       105 non-null    object
 10  10      158 non-null    object
 11  11      34 non-null     object
dtypes: object(12)
memory usage: 15.0+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112 entries, 0 to 111
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       35 non-null     object
 1   1       112 non-null    object
 2   2       112 non-null    object
 3   3       112 non-null    object
 4   4       112 non-null   

# Cleaning-part 1

In [10]:
# when I do data exploration, I find there are two sheets inside Kansas data because of warning when I process sheet 6
# So I move back here to clean this sheet again
sheet6 = sheet['Kansas']
split = sheet6[sheet6[0] == '_2YrD_ src'].index[0]
split

116

In [11]:
# keep Kansas sheet before row 111 and remove rows after 111 to sheet Kansas_2
sheet['Kansas_2'] = sheet['Kansas'].iloc[split:].copy()
sheet['Kansas'] = sheet['Kansas'].iloc[:split].copy()

In [12]:
sheet['Kansas'].tail()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
111,"Dr. Patricio Roman, MD",https://www.healthgrades.com/physician/dr-patr...,"Cosmetic, Plastic & Reconstructive Surgery",This provider has no reviews.,Leave A Review,NaN,2660 Sw 3rd St,"Topeka, KS 66606",57.5 mi,NaN,NaN,NaN,https://www.healthgrades.com/physician/dr-patr...,NaN,NaN,NaN
112,"Dr. Manny Moser, MD",https://www.healthgrades.com/physician/dr-mann...,"Cosmetic, Plastic & Reconstructive Surgery",This provider has no reviews.,Leave A Review,NaN,1001 SW GARFIELD AVE,"Topeka, KS 66604",57.6 mi,NaN,NaN,NaN,https://www.healthgrades.com/physician/dr-mann...,NaN,NaN,NaN
113,"Dr. Lee Daniel, MD",https://www.healthgrades.com/physician/dr-lee-...,"Cosmetic, Plastic & Reconstructive Surgery",This provider has no reviews.,Leave A Review,NaN,710 W 8th St,"Fort Scott, KS 66701",88.7 mi,NaN,NaN,NaN,https://www.healthgrades.com/physician/dr-lee-...,NaN,NaN,NaN
114,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
sheet['Kansas_2'].index

RangeIndex(start=116, stop=144, step=1)

In [14]:
sheet['Kansas_2'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 116 to 143
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       15 non-null     object
 1   1       27 non-null     object
 2   2       27 non-null     object
 3   3       27 non-null     object
 4   4       27 non-null     object
 5   5       27 non-null     object
 6   6       22 non-null     object
 7   7       27 non-null     object
 8   8       27 non-null     object
 9   9       27 non-null     object
 10  10      21 non-null     object
 11  11      19 non-null     object
 12  12      16 non-null     object
 13  13      8 non-null      object
 14  14      7 non-null      object
 15  15      7 non-null      object
dtypes: object(16)
memory usage: 3.6+ KB


In [15]:
# set header of Kansas_2 and reset_index
sheet_ext = sheet['Kansas_2'].copy()
sheet_ext.reset_index(drop=True, inplace=True)
sheet_ext = sheet_ext.rename(columns = sheet_ext.iloc[0]).loc[1:]
sheet_ext.reset_index(drop=True, inplace=True)
# if the line above not working, try below:
# sheet_ext.columns = sheet_ext.iloc[0]
# sheet_ext.drop(0, inplace=True)
# sheet_ext.reset_index(drop=True, inplace=True)
sheet_ext.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 16 columns):
 #   Column                                Non-Null Count  Dtype 
---  ------                                --------------  ----- 
 0   _2YrD_ src                            14 non-null     object
 1   provider-name__lnk                    26 non-null     object
 2   provider-name__lnk href               26 non-null     object
 3   provider-details__specialty           26 non-null     object
 4   sr-only                               26 non-null     object
 5   star-rating__reviews                  26 non-null     object
 6   sr-only 2                             21 non-null     object
 7   location-info__office-loc             26 non-null     object
 8   location-info__office-loc 2           26 non-null     object
 9   location-info__distance               26 non-null     object
 10  feats-of-strength__feat-title         20 non-null     object
 11  feats-of-strength__feat-title 2   

In [16]:
# finish create Kansas_2 sheet
sheet['Kansas_2'] = sheet_ext.copy() # check dataframe copy documentation
# sheet['Kansas_2'] = sheet_ext

In [17]:
sheet['Kansas_2'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 16 columns):
 #   Column                                Non-Null Count  Dtype 
---  ------                                --------------  ----- 
 0   _2YrD_ src                            14 non-null     object
 1   provider-name__lnk                    26 non-null     object
 2   provider-name__lnk href               26 non-null     object
 3   provider-details__specialty           26 non-null     object
 4   sr-only                               26 non-null     object
 5   star-rating__reviews                  26 non-null     object
 6   sr-only 2                             21 non-null     object
 7   location-info__office-loc             26 non-null     object
 8   location-info__office-loc 2           26 non-null     object
 9   location-info__distance               26 non-null     object
 10  feats-of-strength__feat-title         20 non-null     object
 11  feats-of-strength__feat-title 2   

In [18]:
sheet.keys()

dict_keys(['Indiana', 'Missouri', 'Iowa', 'Wisconsin', 'Nebraska', 'Kansas', 'Illinois', 'Kansas_2'])

In [19]:
# try to delete empty rows
drop_lst = {}
names = list(sheet.keys())
for i in range(len(names)):
    for x in range(len(sheet[names[i]])):
        lst = sheet[names[i]].iloc[x, :].isna().to_list()
        for j in range(len(lst)):
            if lst[j] == False:
                break
            if j == len(lst) - 1 and lst[j] == True:
                if i not in drop_lst:
                    drop_lst[i] = [x]
                else:
                    drop_lst[i].append(x)
drop_lst

{5: [0, 4, 114, 115], 6: [3]}

In [20]:
names

['Indiana',
 'Missouri',
 'Iowa',
 'Wisconsin',
 'Nebraska',
 'Kansas',
 'Illinois',
 'Kansas_2']

In [21]:
# delete empty rows
for idx in drop_lst:
    sheet[names[idx]].drop(labels=drop_lst[idx], inplace=True)
    sheet[names[idx]].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 112 entries, 1 to 113
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       112 non-null    object
 1   1       112 non-null    object
 2   2       112 non-null    object
 3   3       112 non-null    object
 4   4       112 non-null    object
 5   5       85 non-null     object
 6   6       112 non-null    object
 7   7       112 non-null    object
 8   8       112 non-null    object
 9   9       76 non-null     object
 10  10      69 non-null     object
 11  11      65 non-null     object
 12  12      28 non-null     object
 13  13      0 non-null      object
 14  14      0 non-null      object
 15  15      0 non-null      object
dtypes: object(16)
memory usage: 14.9+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 283 entries, 0 to 283
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       283 non-null   

In [22]:
# reset_index of Kansas alone
# sheet Kansas is different because the first row is empty, after dropped, header will be row No.1
sheet['Kansas'] = sheet['Kansas'].rename(columns = sheet['Kansas'].iloc[0]).loc[1:]
sheet['Kansas'].drop(1, inplace=True)
sheet['Kansas'].reset_index(drop=True, inplace=True)
sheet['Kansas']

,provider-name__lnk,provider-name__lnk href,provider-details__specialty,sr-only,star-rating__reviews,sr-only 2,location-info__office-loc,location-info__office-loc 2,location-info__distance,feats-of-strength__feat-title,feats-of-strength__feat-title 2,feats-of-strength__feat-title 3,star-rating href,NaN,NaN,NaN
0,"Dr. Clinton Humphrey, MD",https://www.healthgrades.com/physician/dr-clin...,"Cosmetic, Plastic & Reconstructive Surgery","Rated 5 out of 5 stars,",3 ratings,with,4320 Wornall Rd Ste 512,"Kansas City, MO 64111",4.9 mi,Offers Telehealth,Easy scheduling,Employs friendly staff,NaN,NaN,NaN,NaN
1,"Dr. Christian Stallworth, MD",https://www.healthgrades.com/physician/dr-chri...,"Cosmetic, Plastic & Reconstructive Surgery","Rated 5 out of 5 stars,",37 ratings,with,3908 S Noland Rd,"Independence, MO 64055",12.2 mi,Offers Telehealth,Easy scheduling,Employs friendly staff,NaN,NaN,NaN,NaN
2,"Dr. Victor Perez Sr, MD",https://www.healthgrades.com/physician/dr-vict...,"Cosmetic, Plastic & Reconstructive Surgery","Rated 5 out of 5 stars,",17 ratings,with,11532 W 119th St,"Overland Park, KS 66213",14.7 mi,Offers Telehealth,Easy scheduling,Employs friendly staff,NaN,NaN,NaN,NaN
3,"Dr. Levi Young, MD",https://www.healthgrades.com/physician/dr-levi...,"Cosmetic, Plastic & Reconstructive Surgery","Rated 4.5 out of 5 stars,",33 ratings,with,4749 W 134th St,"Leawood, KS 66209",15.7 mi,Offers Telehealth,Easy scheduling,Employs friendly staff,NaN,NaN,NaN,NaN
4,"Dr. Regina Nouhan, MD",https://www.healthgrades.com/physician/dr-regi...,General Hand Surgery,"Rated 4.5 out of 5 stars,",15 ratings,with,4801 W 135th St,"Leawood, KS 66224",15.9 mi,Easy scheduling,Employs friendly staff,Explains conditions well,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,"Dr. Karl Saffo, MD",https://www.healthgrades.com/physician/dr-karl...,"Cosmetic, Plastic & Reconstructive Surgery",This provider has no reviews.,Leave A Review,NaN,301 E KANSAS ST,"Lansing, KS 66043",17 mi,NaN,NaN,NaN,https://www.healthgrades.com/physician/dr-karl...,NaN,NaN,NaN
107,"Dr. Anita Aldrich, MD",https://www.healthgrades.com/physician/dr-anit...,"Cosmetic, Plastic & Reconstructive Surgery",This provider has no reviews.,Leave A Review,NaN,1130 W 4th St Ste 3200,"Lawrence, KS 66044",34.6 mi,NaN,NaN,NaN,https://www.healthgrades.com/physician/dr-anit...,NaN,NaN,NaN
108,"Dr. Patricio Roman, MD",https://www.healthgrades.com/physician/dr-patr...,"Cosmetic, Plastic & Reconstructive Surgery",This provider has no reviews.,Leave A Review,NaN,2660 Sw 3rd St,"Topeka, KS 66606",57.5 mi,NaN,NaN,NaN,https://www.healthgrades.com/physician/dr-patr...,NaN,NaN,NaN
109,"Dr. Manny Moser, MD",https://www.healthgrades.com/physician/dr-mann...,"Cosmetic, Plastic & Reconstructive Surgery",This provider has no reviews.,Leave A Review,NaN,1001 SW GARFIELD AVE,"Topeka, KS 66604",57.6 mi,NaN,NaN,NaN,https://www.healthgrades.com/physician/dr-mann...,NaN,NaN,NaN


In [23]:
# reset_index to other sheets than Kansas
for i in range(len(names)):
    if names[i] == 'Kansas' or names[i] == 'Kansas_2':
        continue
    sheet[names[i]] = sheet[names[i]].rename(columns = sheet[names[i]].iloc[0]).loc[1:]
    sheet[names[i]].reset_index(drop=True, inplace=True)

In [24]:
# check the result
for i in range(len(names)):
    print(sheet[names[i]]['provider-name__lnk'].head())

0          Dr. Joseph Fata, MD
1         Dr. Janet Turkle, MD
2     Dr. Emily Hrisomalos, MD
3    Dr. Christopher Jones, MD
4         Dr. Rachel Scott, MD
Name: provider-name__lnk, dtype: object
0        Dr. Clinton Humphrey, MD
1    Dr. Christian Stallworth, MD
2         Dr. Victor Perez Sr, MD
3              Dr. Levi Young, MD
4           Dr. Regina Nouhan, MD
Name: provider-name__lnk, dtype: object
0    Dr. Benjamin Van Raalte, MD
1       Dr. William Albright, MD
2       Dr. Douglas Henstrom, MD
3          Dr. Aric Eckhardt, MD
4        Dr. Kevin Carpenter, MD
Name: provider-name__lnk, dtype: object
0       Dr. Jason Stubbs, DO
1        Dr. Gary Kronen, MD
2    Dr. Anh-Tuan Truong, MD
3           Dr. John Kim, MD
4     Dr. Allan Parungao, MD
Name: provider-name__lnk, dtype: object
0     Dr. Gerard Stanley, MD
1       Dr. David Finkle, MD
2         Dr. Nagi Ayoub, MD
3    Dr. Frank Soto Leon, MD
4      Dr. John Harbison, MD
Name: provider-name__lnk, dtype: object
0        Dr. Clinton

In [25]:
# save the cleaned data within one excel file: cleaned-v1
with pd.ExcelWriter('/Users/jennyhuang/Documents/jobs/DBC-AI Launch/project2-MDdataCleaning/cleaned-v1.xlsx') as writer:
    for i in range(len(names)):
        sheet[names[i]].to_excel(writer, sheet_name=str(i)+' '+names[i], index=False)

# Cleaning-part 2

In [1]:
df = pd.read_excel('/Users/jennyhuang/Documents/jobs/DBC-AI Launch/project2-MDdataCleaning/cleaned-v1.xlsx')

NameError: name 'pd' is not defined

In [4]:
file = pd.ExcelFile('/Users/jennyhuang/Documents/jobs/DBC-AI Launch/project2-MDdataCleaning/cleaned-v1.xlsx')

In [5]:
names = file.sheet_names
names

['0 Indiana',
 '1 Missouri',
 '2 Iowa',
 '3 Wisconsin',
 '4 Nebraska',
 '5 Kansas',
 '6 Illinois',
 '7 Kansas_2']

In [6]:
sheet = {}
for i in range(len(names)):
    sheet[names[i]] = file.parse(i, header = None)

In [7]:
for i in range(len(names)):
    vars()['sheet'+str(i + 1)] = sheet[names[i]].rename(columns=sheet[names[i]].iloc[0]).loc[1:] # let the string to become variable
    vars()['sheet'+str(i + 1)].reset_index(drop=True, inplace=True)
    vars()['sheet'+str(i + 1)].info()# sheet3: lowa doesn't have sr-only column

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158 entries, 0 to 157
Data columns (total 12 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   provider-name__lnk               157 non-null    object
 1   provider-details__specialty      157 non-null    object
 2   sr-only                          157 non-null    object
 3   star-rating__reviews             157 non-null    object
 4   sr-only 2                        124 non-null    object
 5   location-info__office-loc        157 non-null    object
 6   location-info__office-loc 2      157 non-null    object
 7   feats-of-strength__feat-title    120 non-null    object
 8   feats-of-strength__feat-title 2  106 non-null    object
 9   feats-of-strength__feat-title 3  104 non-null    object
 10  location-info__distance 2        157 non-null    object
 11  star-rating href                 33 non-null     object
dtypes: object(12)
memory usage: 14.9+ KB

In [8]:
for i in range(1, 9):
    print(len(vars()['sheet'+str(i)]) == len(vars()['sheet'+str(i)]['provider-name__lnk']))
    # there's no duplicate name in all sheets

True
True
True
True
True
True
True
True


In [9]:
# general clean for the rating column
for i in range(len(names)):
    cur_sheet = vars()['sheet'+str(i + 1)]
    if i == 2:
        continue
    for j in range(len(cur_sheet['sr-only'])):
        new = re.findall("\d+\.\d+", str(cur_sheet['sr-only'].iloc[j]))
        if len(new) == 0:
            if len(re.findall("\d+", str(cur_sheet['sr-only'].iloc[j]))) == 0:
                cur_sheet['sr-only'].iloc[j] = -1
            else:
                new.append(float(re.findall("\d+", str(cur_sheet['sr-only'].iloc[j]))[0]))
                cur_sheet['sr-only'].iloc[j] = new[0]
        else:
            cur_sheet['sr-only'].iloc[j] = new[0]

In [10]:
# clean the star-rating__reviews column to review_number column, result shows that sheet3 doesn't have star-rating__review
for i in range(len(names)):
    cur_sheet = vars()['sheet'+str(i + 1)]
    review_number = []
    try:
        for j in range(len(cur_sheet['star-rating__reviews'])):
            lst = re.findall("\d+", str(cur_sheet['star-rating__reviews'].iloc[j]))
            if len(lst) != 0:
                review_number.append(int(lst[0]))
            else:
                review_number.append(-1)
        cur_sheet['review_number'] = review_number
    except:
        print('Error Occured when processing sheet' + str(i + 1))

Error Occured when processing sheet3


In [11]:
type(sheet3['provider-details__specialty'].iloc[:, 0].unique())

numpy.ndarray

In [12]:
sheet3.columns

Index(['_2YrD_ src', 'provider-name__lnk', 'provider-name__lnk href',
       'provider-details__specialty', 'provider-details__specialty',
       'provider-details__specialty', 'sr-only 2', 'location-info__office-loc',
       'location-info__office-loc 2', 'location-info__distance',
       'feats-of-strength__feat-title', 'feats-of-strength__feat-title 2',
       'feats-of-strength__feat-title 3'],
      dtype='object')

In [13]:
sheet3.columns = ['_2YrD_ src', 'provider-name__lnk', 'provider-name__lnk href',
       'provider-details__specialty1', 'provider-details__specialty2',
       'provider-details__specialty3', 'sr-only 2', 'location-info__office-loc',
       'location-info__office-loc 2', 'location-info__distance',
       'feats-of-strength__feat-title', 'feats-of-strength__feat-title 2',
       'feats-of-strength__feat-title 3']

In [14]:
# # clean sheet3's provider-details__specialty column in general
# for i in range(3):
#     cur_col = sheet3[f'provider-details__specialty{str(i + 1)}']
#     vars()['array'+str(i + 1)] = cur_col.unique()
#     vars()['dic'+str(i + 1)] = {}
#     vars()['provider_detail' + str(i + 1)] = []
#     cur_array = vars()['array'+str(i + 1)]
#     cur_dic = vars()['dic'+str(i + 1)]
#     cur_pd = vars()['provider_detail' + str(i + 1)]
#     for j in range(len(cur_array)):
#         cur_dic[cur_array[j]] = j
#     for k in range(len(cur_col)):
#         ori = cur_col.iloc[k]
#         cur_pd.append(cur_dic[ori])
#     cur_col = cur_pd
#     print(cur_dic)

In [15]:
# # categorized the provider-details__specialty column to provider_detail column except for sheet 3
# for i in range(len(names)):
#     cur_sheet = vars()['sheet'+str(i + 1)]
#     dic = {}
# #     print(type(cur_sheet['provider-details__specialty'])) # sheet3 has 3 'provider-details__specialty' columns
#     if i == 2:
#         continue
#     else:
#         array = cur_sheet['provider-details__specialty'].unique()
#         try:
#             for j in range(len(array)):
#                 dic[array[j]] = j
#             provider_detail = []
#             for x in range(len(cur_sheet['provider-details__specialty'])):
#                 ori = cur_sheet['provider-details__specialty'].iloc[i]
#                 provider_detail.append(dic[ori])
#             cur_sheet['provider_detail'] = provider_detail
#         except:
#             print('Error Occured when processing sheet' + str(i + 1))

In [16]:
# test = set([1, 2, 3, 4, 5, 6])
# add = set([5, 6, 7, 8, 9])
# test.update(add)
# test

In [14]:
## version 2- combine all the unique variables then categorize it
## put every provider-details__specialty of all sheets into one set
provider_detail = set()
for i in range(len(names)):
    cur_sheet = vars()['sheet'+str(i + 1)]
    if i == 2:
        for j in range(3):
            cur_col = sheet3[f'provider-details__specialty{str(i + 1)}']
            provider_detail.update(set(cur_col.unique()))
    else:
        provider_detail.update(cur_sheet['provider-details__specialty'].unique())
        
## map the provider_detail set into categorical variable
provider_detail_lst = list(provider_detail)
dic = {}
for i in range(len(provider_detail_lst)):
    dic[provider_detail_lst[i]] = i
    
## map the dic into provider_detail column
for i in range(len(names)):
    cur_sheet = vars()['sheet'+str(i + 1)]
    provider_lst = []
    if i == 2:
        for j in range(3):
            cur_col = sheet3[f'provider-details__specialty{str(i + 1)}']
            provider_lst = []
            for k in range(len(cur_col)):
                provider_lst.append(dic[cur_col.iloc[k]])
            sheet3[f'provider_detail{i + 1}'] = provider_lst
    else:
        for j in range(len(cur_sheet['provider-details__specialty'])):
            provider_lst.append(dic[cur_sheet['provider-details__specialty'].iloc[j]])
        cur_sheet['provider_detail'] = provider_lst

In [15]:
dic

{nan: 0,
 'General Surgery': 1,
 'Nursing (Nurse Practitioner)': 2,
 'Mohs Micrographic Surgery': 3,
 'Urogynecology & Reconstructive Pelvic Surgery': 4,
 'Oculoplastic Surgery': 5,
 'Ophthalmology': 6,
 'Physician Assistant (PA)': 7,
 'Dermatology (Nurse Practitioner)': 8,
 'Orthopedic Hand Surgery': 9,
 'Family Medicine': 10,
 'Oral & Maxillofacial Surgery': 11,
 'Ear, Nose, and Throat': 12,
 'Dermatology': 13,
 'Urology': 14,
 'Bariatric Surgery': 15,
 'Phlebology': 16,
 'Cosmetic Medicine': 17,
 'Emergency Medicine': 18,
 'Hair Transplant Surgery': 19,
 'General Hand Surgery': 20,
 'Dentistry': 21,
 'Speech-Language Pathology': 22,
 'Breast Surgery': 23,
 'Cosmetic, Plastic & Reconstructive Surgery': 24,
 'Pediatric Plastic Surgery': 25,
 'Dermatologic Surgery': 26,
 'Obstetrics & Gynecology': 27,
 'Dermatopathology': 28,
 'Head & Neck Surgical Oncology': 29}

In [30]:
len(dic)
dic.values()

dict_values([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29])

In [18]:
# # try all sheets together
# # change 'feats-of-strength__feat-title' column to feats column
# # change 'feats-of-strength__feat-title 2' column to feats1 column
# # change 'feats-of-strength__feat-title 3' column to feats2 column
# for i in range(1, 9):
#     cur_sheet = vars()['sheet' + str(i)]
#     cur_sheet['feats-of-strength__feat-title'].fillna("NaN", inplace=True)
#     cur_sheet['feats-of-strength__feat-title 2'].fillna("NaN", inplace=True)
#     cur_sheet['feats-of-strength__feat-title 3'].fillna("NaN", inplace=True)
#     array = cur_sheet['feats-of-strength__feat-title'].unique()
#     array2 = cur_sheet['feats-of-strength__feat-title 2'].unique()
#     array3 = cur_sheet['feats-of-strength__feat-title 3'].unique()
#     lst1 = list(set(array))
#     lst2 = list(set(array2))
#     lst3 = list(set(array3))
#     for j in range(3):
#         cur_dic = {}
#         cur_lst = vars()['lst'+str(j + 1)]
#         for k in range(len(cur_lst)):
#             cur_dic[cur_lst[k]] = k
#         vars()['dic' + str(i) + '_' + str(j)] = cur_dic
# #         print(vars()['dic' + str(i) + '_' + str(j)]) # just for check
#         # i start from 1, j start from 0

In [31]:
# try all sheets together
# combine 'feats-of-strength__feat-title' in all sheets and map it to number
# combine 'feats-of-strength__feat-title 2' in all sheets and map it to number
# combine 'feats-of-strength__feat-title 3' in all sheets and map it to number
lst1 = set()
lst2 = set()
lst3 = set()
combine_array = []
for i in range(1, 9):
    cur_sheet = vars()['sheet' + str(i)]
    cur_sheet['feats-of-strength__feat-title'].fillna("NaN", inplace=True)
    cur_sheet['feats-of-strength__feat-title 2'].fillna("NaN", inplace=True)
    cur_sheet['feats-of-strength__feat-title 3'].fillna("NaN", inplace=True)
    array = cur_sheet['feats-of-strength__feat-title'].unique()
    array2 = cur_sheet['feats-of-strength__feat-title 2'].unique()
    array3 = cur_sheet['feats-of-strength__feat-title 3'].unique()
    lst1.update(set(array))
    lst2.update(set(array2))
    lst3.update(set(array3))
    
lst1 = list(lst1)
lst2 = list(lst2)
lst3 = list(lst3)
for j in range(3):
    cur_dic = {}
    cur_lst = vars()['lst'+str(j + 1)]
    for k in range(len(cur_lst)):
        cur_dic[cur_lst[k]] = k
    vars()['dic' + str(j)] = cur_dic
    print(vars()['dic' + str(j)]) # just for check

{'NaN': 0, 'Employs friendly staff': 1, 'Explains conditions well': 2, 'Offers Telehealth': 3, 'Low wait times': 4, 'Easy scheduling': 5, 'Patients found trustworthy': 6}
{'NaN': 0, 'Employs friendly staff': 1, 'Explains conditions well': 2, 'Low wait times': 3, 'Easy scheduling': 4, 'Patients found trustworthy': 5}
{'Employs friendly staff': 0, 'NaN': 1, 'Explains conditions well': 2, 'Low wait times': 3, 'Patients found trustworthy': 4}


In [32]:
for i in range(3):
    print(len(f'dic{i}'))

4
4
4


In [20]:
# check whether the length are different
for i in range(1, 9):
    cur_sheet = vars()['sheet'+str(i)]
    if len(cur_sheet) != len(cur_sheet['feats-of-strength__feat-title']):
        print(f'sheet{i}' + 'len(cur_sheet) != len(cur_sheet[feats-of-strength__feat-title])')
    if len(cur_sheet) != len(cur_sheet['feats-of-strength__feat-title 2']):
        print(f'sheet{i}' + 'len(cur_sheet) != len(cur_sheet[feats-of-strength__feat-title 2])')
    if len(cur_sheet) != len(cur_sheet['feats-of-strength__feat-title 3']):
        print(f'sheet{i}' + 'len(cur_sheet) != len(cur_sheet[feats-of-strength__feat-title 3])')

In [21]:
for i in range(1, 9):
    feats, feats1, feats2 = [], [], []
    cur_sheet = vars()['sheet'+str(i)]
    for j in range(len(cur_sheet)):
        feats.append(vars()['dic0'][cur_sheet['feats-of-strength__feat-title'].iloc[j]])
        feats1.append(vars()['dic1'][cur_sheet['feats-of-strength__feat-title 2'].iloc[j]])
        feats2.append(vars()['dic2'][cur_sheet['feats-of-strength__feat-title 3'].iloc[j]])
    cur_sheet['feats'] = feats
    cur_sheet['feats1'] = feats1
    cur_sheet['feats2'] = feats2
#     print(cur_sheet.head()) # just for check

In [22]:
# change the column name: location-info__distance 2 to location-info__distance in sheet1
sheet1.columns
sheet1.columns = ['provider-name__lnk', 'provider-details__specialty', 'sr-only',
       'star-rating__reviews', 'sr-only 2', 'location-info__office-loc',
       'location-info__office-loc 2', 'feats-of-strength__feat-title',
       'feats-of-strength__feat-title 2', 'feats-of-strength__feat-title 3',
       'location-info__distance', 'star-rating href', 'review_number',
       'provider_detail', 'feats', 'feats1', 'feats2']

In [23]:
# clean the location-info__distance column to distance column
for i in range(1, 9):
    cur_sheet = vars()['sheet'+str(i)]
    for j in range(len(cur_sheet)):
        lst = re.findall("\d+\.\d+", str(cur_sheet['location-info__distance'].iloc[j]))
        if len(lst) == 0:
            try:
                cur_sheet['location-info__distance'].iloc[j] = re.findall("\d+", str(cur_sheet['location-info__distance'].iloc[j]))[0]
            except:
                print('sheet'+str(i) + ', ' + 'line' + str(j))
        else:
            cur_sheet['location-info__distance'].iloc[j] = lst[0]
#     print(cur_sheet['location-info__distance'].head()) # just for check

/var/folders/20/h9hzw4qs64321s82n6w8zjyr0000gn/T/ipykernel_8839/801636956.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cur_sheet['location-info__distance'].iloc[j] = lst[0]
/var/folders/20/h9hzw4qs64321s82n6w8zjyr0000gn/T/ipykernel_8839/801636956.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cur_sheet['location-info__distance'].iloc[j] = re.findall("\d+", str(cur_sheet['location-info__distance'].iloc[j]))[0]


sheet1, line157
sheet4, line661
sheet8, line2


In [24]:
sheet1.iloc[157]

provider-name__lnk                                                               NaN
provider-details__specialty                                                      NaN
sr-only                                                                           -1
star-rating__reviews                                                             NaN
sr-only 2                                                                        NaN
location-info__office-loc                                                        NaN
location-info__office-loc 2                                                      NaN
feats-of-strength__feat-title                                                    NaN
feats-of-strength__feat-title 2                                                  NaN
feats-of-strength__feat-title 3                                                  NaN
location-info__distance                                                          NaN
star-rating href                   https://www.healthgrades.com/p

In [25]:
sheet4.iloc[661]

_2YrD_ src                                                                       NaN
provider-name__lnk                                                               NaN
provider-name__lnk href                                                          NaN
provider-details__specialty                                                      NaN
sr-only                                                                           -1
star-rating__reviews                                                             NaN
sr-only 2                                                                        NaN
location-info__office-loc                                                        NaN
location-info__office-loc 2                                                      NaN
location-info__distance                                                          NaN
feats-of-strength__feat-title                                                    NaN
feats-of-strength__feat-title 2                                  

In [26]:
sheet8.iloc[2]

_2YrD_ src                                                   NaN
provider-name__lnk                                           NaN
provider-name__lnk href                                      NaN
provider-details__specialty                                  NaN
sr-only                                                       -1
star-rating__reviews                                         NaN
sr-only 2                                                    NaN
location-info__office-loc                                    NaN
location-info__office-loc 2                                  NaN
location-info__distance                                      NaN
feats-of-strength__feat-title                                NaN
feats-of-strength__feat-title 2                              NaN
feats-of-strength__feat-title 3                              NaN
button                                  Find a Telehealth Doctor
button href                                                  NaN
card-sponsor__sponsor-inf

In [27]:
# save 7 sheets into seperate csv file
src_path  = "/Users/jennyhuang/Documents/jobs/DBC-AI Launch/project2-MDdataCleaning/data-explore/FileName.csv"
for i in range(1, 9):
    p = Path(src_path).parent.joinpath(f"sheet{str(i)}.csv")
    vars()['sheet' + str(i)].to_csv(p, index=False)